In [5]:
import json
import time
import base64
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction
sys.path.append(os.path.abspath(os.path.join('../scripts/')))
from passwords import api,algo_online

In [6]:
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print('Waiting for confirmation')
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print('Transaction confirmed in round', txinfo.get('confirmed-round'))
    return txinfo


In [26]:
# Setup HTTP client w/guest key provided by PureStake
algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
algod_token = ""
api_key = api['key']
headers = {
    "X-API-Key": api_key,
}



In [27]:
algodclient = algod.AlgodClient(algod_token, algod_address, headers)
# get suggested parameters from Algod
params = algodclient.suggested_params()

In [32]:



# get suggested parameters from Algod
params = algodclient.suggested_params()

gh = params.gh
first_valid_round = params.first
last_valid_round = params.last
fee = params.min_fee
send_amount = 10

existing_account = 'YLH4K73F2SSTH7F77CBWDJZO3W63WXREXD7EVUUOJ6GZM5CFF5KW3OUNEA'
send_to_address = 'AEC4WDHXCDF4B5LBNXXRTB3IJTVJSWUZ4VJ4THPU2QGRJGTA3MIDFN3CQA'

# Create and sign transaction
tx = transaction.PaymentTxn(existing_account, fee, first_valid_round, last_valid_round, gh, send_to_address,
                            send_amount, flat_fee=True)
signed_tx = tx.sign(algo_online['my_private_key'])

try:
    tx_confirm = algodclient.send_transaction(signed_tx)
    print('Transaction sent with ID', signed_tx.transaction.get_txid())
    wait_for_confirmation(algodclient, txid=signed_tx.transaction.get_txid())
except Exception as e:
    print(e)

Transaction sent with ID 6LZNZ435EI7KS224NIDJNZIJANJ5TZCJ4P4IGTT2XSHC6OQKNGVQ
Waiting for confirmation
Transaction confirmed in round 24426757
